# Library

In [30]:
import requests
from bs4 import BeautifulSoup as bs

import time, os, random
import pandas as pd
import numpy as np
from datetime import datetime

from selenium import webdriver
import chromedriver_autoinstaller as ca

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

## chrome driver 설치

In [2]:
# USB error 메세지 발생 해결을 위한 코드
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])

In [3]:
# 현재 크롬 버전 확인
chrome_ver = ca.get_chrome_version().split('.')[0]
chrome_ver

'131'

In [4]:
# # 크롬 드라이버 확인 및 설치(처음 한번만 실행)
# ca.install(True)

# 페이지 접속

In [ ]:
# url = 'https://section.blog.naver.com/BlogHome.naver?directoryNo=0&currentPage=1&groupId=0' # 네이버 블로그 홈

## requests 테스트
- 페이지 접속 가능 여부확인
    - 가능할 경우 출력 : <Response [200]>

In [7]:
# req = requests.get(url)
# print(req)

# 한글 깨짐 해결 코드
# # html = req.content.decode('utf-8') # 한글 깨짐 해결
# # soup = bs(html, 'html.parser')

# soup = bs(req.text, 'html.parser')
# soup.title.text

<Response [200]>


'네이버 블로그'

In [4]:
# request로 수집 에러
keyword = '아이폰 16' # 검색어
page_num = 1 # 페이지 번호
rangetype = 'ALL' # 검색 범위
orderby = 'sim' # 정렬 순서

keyword_url = f'https://section.blog.naver.com/Search/Post.naver?pageNo={page_num}&rangeType={rangetype}&orderBy={orderby}&keyword={keyword}'
req = requests.get(keyword_url, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36'})
print(req)
soup = bs(req.text, 'html.parser')
soup.title.text

<Response [200]>


'네이버 블로그'

In [6]:
# area_list_search = soup.select_one('div.area_list_search')
# list_search_post = area_list_search.select('div.list_search_post')
# list_search_post[0]

## keyword 입력

In [9]:
keyword = '아이폰 16' # 검색어

## 탭 선택(글, 블로그)

In [10]:
tab_option = '글' # 포스트, 블로그 선택
page_num = 1 # 페이지 번호
rangetype = 'ALL' # 검색 범위
orderby = 'sim' # 정렬 순서

if tab_option == '글':
    keyword_url = f'https://section.blog.naver.com/Search/Post.naver?pageNo={page_num}&rangeType={rangetype}&orderBy={orderby}&keyword={keyword}'
elif tab_option == '블로그':
    keyword_url = f'https://section.blog.naver.com/Search/Blog.naver?pageNo={page_num}&orderBy={orderby}&keyword={keyword}'

## selenium 작동

In [11]:
# 웹드라이버 실행
driver = webdriver.Chrome(options=options)

In [12]:
driver.get(keyword_url)
driver.implicitly_wait(10) # 페이지 로드 될 때까지 기다리지만 로드 되는 순간 종료
# driver.maximize_window() # 브라우져 창 최대화

In [ ]:
# keyword = '아이폰 16' # 검색어
# keyword_input_xpath = '//*[@id="header"]/div[1]/div/div[2]/form/fieldset/div/input' # 검색창 xpath
# keyword_input_box = driver.find_element(By.XPATH, keyword_input_xpath)
# keyword_input_box.send_keys(keyword)
# time.sleep(random.uniform(1, 2))
# keyword_input_box.send_keys(Keys.ENTER)
# driver.implicitly_wait(10)

# 정보 수집

### 🔧 글 선택
- 참고 링크 : https://section.blog.naver.com/Search/Post.naver?pageNo={page_num}&rangeType=ALL&orderBy=sim&keyword={keyword}
- 참고 링크 : https://section.blog.naver.com/Search/Post.naver?pageNo=1&rangeType=WEEK&orderBy=sim&startDate=2025-01-03&endDate=2025-01-10&keyword=아이폰 16
- 참고 링크 : https://section.blog.naver.com/Search/Post.naver?pageNo=1&rangeType=PERIOD&orderBy=sim&startDate=2025-01-01&endDate=2025-01-13&keyword=아이폰 16
- [⭕] rangeType 옵션(기간)
    - 기간전체 : &rangeType=ALL
    - 최근 1주 : &rangeType=WEEK
    - 최근 1개월 : &rangeType=MONTH
    - 기간 입력 : &rangeType=PERIOD&startDate=YYYY-mm-dd&endDate=YYYY-mm-dd

- [⭕] orderBy 옵션(관련도순, 최신순)
    - 관련도순 : &orderBy=sim
    - 최신순 : &orderBy=recentdate

- 수집할 페이지 옵션 추가
- 수집할 페이지가 실제 페이지 수보다 많을 경우 예외 처리

In [13]:
page = driver.page_source
soup = bs(page, 'html.parser')
print(soup.title.text)

네이버 블로그


In [14]:
area_list_search = soup.select_one('div.area_list_search')
list_search_post = area_list_search.select('div.list_search_post')
list_search_post[0]

<div anchor-text="content" bg-move-scroll-on-pop-state="" class="list_search_post" ng-repeat="post in postSearchCtrl.resultList">
<div bg-article-multi-img-style="" class="item multi_pic">
<div class="info_post" ng-class="{only_post: post.thumbnails.length == 0}">
<div class="desc">
<a bg-nclick="srs*l.text" class="desc_inner" href="https://blog.naver.com/mlifeso/223722026004" ng-href="https://blog.naver.com/mlifeso/223722026004" target="_blank">
<strong class="title_post">
<!-- ngIf: post.titleHeadColor == '#27C400' -->
<!-- ngIf: post.titleHeadColor == '#FF6565' -->
<span class="title" ng-bind-html="post.title"><strong class="search_keyword">아이폰16</strong> 성능 15와 비교 및 할인팁</span>
</strong>
</a><!-- ngIf: post.contents --><a bg-nclick="srs*l.text" class="text" href="https://blog.naver.com/mlifeso/223722026004" ng-bind-html="post.contents" ng-href="https://blog.naver.com/mlifeso/223722026004" ng-if="post.contents" target="_blank"><strong class="search_keyword">아이폰16</strong> 성능이 궁금하다면 이

In [15]:
# 글 제목
post_num = 0
title = list_search_post[post_num].select_one('span.title').text
title

'아이폰16 성능 15와 비교 및 할인팁'

In [16]:
# 블로그 내용
text = list_search_post[post_num].select_one('a.text').text
text

'아이폰16 성능이 궁금하다면 이번 포스팅을 주목해 주세요. 전작 15 시리즈와 비교했을 때 성능이 얼마나 업그레이드되었는지 비교해 보는 시간을 가져볼게요. 이번 아이폰16 시리즈는 전기종 가격이 동결되었는데요. 여전히 비싼 가격이라는 건 변함없는 사실입니다. 최대 보조금을 받아... '

In [17]:
# 글 작성자
name_author = list_search_post[post_num].select_one('em.name_author').text
name_author

'곰순이'

In [18]:
# 블로그 이름
name_blog = list_search_post[post_num].select_one('span.name_blog').text
name_blog

'곰순이'

In [19]:
# 글 작성 날짜
date = list_search_post[post_num].select_one('span.date').text
date

'2025. 1. 10.'

In [20]:
# 글 링크
desc_inner = list_search_post[post_num].select_one('a.desc_inner')['href']
desc_inner

'https://blog.naver.com/mlifeso/223722026004'

In [21]:
list_search_post[1]

<div anchor-text="content" bg-move-scroll-on-pop-state="" class="list_search_post" ng-repeat="post in postSearchCtrl.resultList">
<div bg-article-multi-img-style="" class="item multi_pic">
<div class="info_post" ng-class="{only_post: post.thumbnails.length == 0}">
<div class="desc">
<a bg-nclick="srs*l.text" class="desc_inner" href="https://blog.naver.com/cjfgns2/223720560043" ng-href="https://blog.naver.com/cjfgns2/223720560043" target="_blank">
<strong class="title_post">
<!-- ngIf: post.titleHeadColor == '#27C400' -->
<!-- ngIf: post.titleHeadColor == '#FF6565' -->
<span class="title" ng-bind-html="post.title"><strong class="search_keyword">아이폰16</strong> 프로 디자인 보조금 꿀팁까지</span>
</strong>
</a><!-- ngIf: post.contents --><a bg-nclick="srs*l.text" class="text" href="https://blog.naver.com/cjfgns2/223720560043" ng-bind-html="post.contents" ng-href="https://blog.naver.com/cjfgns2/223720560043" ng-if="post.contents" target="_blank">오늘은 <strong class="search_keyword">아이폰16</strong> 프로 디자인과

In [22]:
# 글 제목
post_num = 1
title = list_search_post[post_num].select_one('span.title').text
title

'아이폰16 프로 디자인 보조금 꿀팁까지'

In [23]:
# 블로그 내용
text = list_search_post[post_num].select_one('a.text').text
text

'오늘은 아이폰16 프로 디자인과 달라진 주요 스펙을 정리해 보고, 가격 할인을 받을 수 있는 방법까지 총정리해 보는 시간을 가져볼게요. 서두는 이만하고 바로 본론으로 넘어가 자세히 알아보도록 하겠습니다. <목차> 1.아이폰16 프로 스펙 및 디자인 2.아이폰16 프로 컬러 및 가격 3. 할인 크게... '

In [24]:
# 글 작성자
name_author = list_search_post[post_num].select_one('em.name_author').text
name_author

'얌못'

In [25]:
# 블로그 이름
name_blog = list_search_post[post_num].select_one('span.name_blog').text
name_blog

'경제꿈나무'

In [26]:
# 글 작성 날짜
date = list_search_post[post_num].select_one('span.date').text
date

'2025. 1. 9.'

In [27]:
# 글 링크
text_link = list_search_post[post_num].select_one('a.desc_inner')['href']
text_link

'https://blog.naver.com/cjfgns2/223720560043'

In [28]:
area_list_search = soup.select_one('div.area_list_search')
list_search_post = area_list_search.select('div.list_search_post')

title_list = []
text_list = []
name_author_list = []
name_blog_list = []
date_list = []
text_link_list = []

for post in list_search_post:
    title = post.select_one('span.title').text
    text = post.select_one('a.text').text
    name_author = post.select_one('em.name_author').text
    name_blog = post.select_one('span.name_blog').text
    date = post.select_one('span.date').text
    text_link = post.select_one('a.desc_inner')['href']
    # print(title, text, name_author, name_blog, text_link)

    title_list.append(title)
    text_list.append(text)
    name_author_list.append(name_author)
    name_blog_list.append(name_blog)
    date_list.append(date)
    text_link_list.append(text_link)
print(len(title_list), len(text_list), len(name_author_list), len(name_blog_list), len(date_list), len(text_link_list))

7 7 7 7 7 7


In [58]:
driver.quit()

In [36]:
# 웹드라이버 실행
driver = webdriver.Chrome(options=options)

In [34]:
start = 5
end = 3
start, end = end, start
print(start, end)

3 5


In [37]:
keyword = '아이폰 16' # 검색어
tab_option = '글' # 포스트, 블로그 선택
rangetype = 'PERIOD' # 검색 범위(ALL, WEEK, MONTH, PERIOD)
orderby = 'sim' # 정렬 순서(sim, recentdate)

# rangetype이 PERIOD인 경우 시작일과 종료일 설정 
startdate = '2025-01-01' # 형식: YYYY-mm-dd(예. 2025-01-01)
enddate = '2025-01-10' # 형식: YYYY-mm-dd(예. 2025-01-01)
current_date = datetime.today().strftime('%Y-%m-%d')
if startdate > enddate:
    startdate, enddate = enddate, startdate # 시작일이 종료일보다 크면 종료일로 변경
if startdate > current_date:
    startdate, enddate = current_date, current_date # 시작일이 현재 날짜보다 크면 현재 날짜로 변경
if enddate > current_date:
    enddate = current_date # 종료일이 현재 날짜보다 크면 현재 날짜로 변경

title_list = []
text_list = []
name_author_list = []
name_blog_list = []
date_list = []
post_link_list = []

for page_num in range(1, 10):
    if tab_option == '글':
        if rangetype == 'PERIOD':
            keyword_url = f'https://section.blog.naver.com/Search/Post.naver?pageNo={page_num}&rangeType={rangetype}&orderBy={orderby}&&startDate={startdate}&endDate={enddate}&keyword={keyword}'
        else:
            keyword_url = f'https://section.blog.naver.com/Search/Post.naver?pageNo={page_num}&rangeType={rangetype}&orderBy={orderby}&keyword={keyword}'
    elif tab_option == '블로그':
        keyword_url = f'https://section.blog.naver.com/Search/Blog.naver?pageNo={page_num}&orderBy={orderby}&keyword={keyword}'


    driver.get(keyword_url)
    driver.implicitly_wait(10) # 페이지 로드 될 때까지 기다리지만 로드 되는 순간 종료
    time.sleep(random.uniform(1, 3))

    page = driver.page_source
    soup = bs(page, 'html.parser')
    # print(soup.title.text)
    
    area_list_search = soup.select_one('div.area_list_search')
    list_search_post = area_list_search.select('div.list_search_post')

    for post in list_search_post:
        title = post.select_one('span.title').text
        text = post.select_one('a.text').text
        name_author = post.select_one('em.name_author').text
        name_blog = post.select_one('span.name_blog').text
        date = post.select_one('span.date').text
        post_link = post.select_one('a.desc_inner')['href']

        title_list.append(title)
        text_list.append(text)
        name_author_list.append(name_author)
        name_blog_list.append(name_blog)
        date_list.append(date)
        post_link_list.append(post_link)

print(len(title_list), len(text_list), len(name_author_list), len(name_blog_list), len(date_list), len(post_link_list))

63 63 63 63 63 63


In [38]:
post_dict = {
    'title': title_list,
    'text': text_list,
    'name_author': name_author_list,
    'name_blog': name_blog_list,
    'date': date_list,
    'post_link': post_link_list
}
post_df = pd.DataFrame(post_dict)
post_df

,title,text,name_author,name_blog,date,post_link
0,아이폰16 성능 15와 비교 및 할인팁,아이폰16 성능이 궁금하다면 이번 포스팅을 주목해 주세요. 전작 15 시리즈와 비교...,곰순이,곰순이,2025. 1. 10.,https://blog.naver.com/mlifeso/223722026004
1,아이폰16 프로 디자인 보조금 꿀팁까지,"오늘은 아이폰16 프로 디자인과 달라진 주요 스펙을 정리해 보고, 가격 할인을 받을...",얌못,경제꿈나무,2025. 1. 9.,https://blog.naver.com/cjfgns2/223720560043
2,아이폰 16프로 내추럴 티타늄 언박싱 구매후기,아이폰 16프로 내추럴 티타늄 언박싱 구매후기 남편돈 남편산 구매후기입니다 저는 아...,grace,GRACE's lov diary,2025. 1. 7.,https://blog.naver.com/dmsp3535/223717372581
3,아이폰16프로 256 가격 성지 할인,최신 스마트폰 중에서도 아이폰16 프로는 강력한 성능과 세련된 디자인으로 많은 이들...,테크매니아,테크톡톡,2025. 1. 3.,https://blog.naver.com/sexyleep/223712531240
4,"SK 아이폰 16 프로 가격, 이렇게 싸게 살 수 있다고?","안녕하세요~ 오늘은 많은 분들이 기다리고 계신 최신 스마트폰, SK 아이폰 16 프...",marktup,marktup,2025. 1. 3.,https://blog.naver.com/marktup/223713528440
...,...,...,...,...,...,...
58,아이폰16 성지 출고가보다 싸게 구매하기,"오늘은 아이폰 16의 스펙과 출고가, 그리고 아이폰16 성지를 활용해 저렴하게 구매...",DevelopPharmacy,DevelopPharmacy,2025. 1. 7.,https://blog.naver.com/pangyolee/223717746406
59,아이폰16 프로 256 가격 전작 15프로 차이,애플의 아이폰16 시리즈는 최근 몇 년간 출시된 모델 중 가장 적은 인기를 얻었다는...,해모수,"넓은 세상으로,,,,",2025. 1. 4.,https://blog.naver.com/owen1108/223713961050
60,아이폰16 구매 제일 저렴하게,아이폰16 구매를 앞두고 있다면 이번 포스팅을 참고해 주세요. 아이폰16의 달라진 ...,곰순이,곰순이,2025. 1. 8.,https://blog.naver.com/mlifeso/223718201181
61,아이폰16 블루 컬러와 성지 활용 실속 구매 가이드,아이폰16 블루는 전작인 아이폰 15 블루보다 더 진한 색감과 세련된 디자인으로 많...,DevelopPharmacy,DevelopPharmacy,2025. 1. 4.,https://blog.naver.com/pangyolee/223714772074


In [39]:
new = post_df.drop_duplicates()
new.shape

(63, 6)

In [56]:
# 현재 날짜
current_date = datetime.today().strftime('%Y%m%d')
current_date

'20250113'

In [57]:
# 현재 경로 확인
code_path = os.getcwd().replace('\\', '/')
code_path

'c:/Users/kbjoo/Documents/Google_Drive/GitHub/my_projects/crawling/naver'

In [ ]:
# 수집한 파일 저장할 폴더 생성
crawled_folder_path = os.path.join(code_path, 'crawled_data', 'naver_blog', current_date)
os.makedirs(crawled_folder_path, exist_ok=True)

In [70]:
current_datetime = datetime.today().strftime('%Y%m%d_%p_%I%M%S')
current_datetime

'20250113_AM_113519'

In [ ]:
# 저장할 파일 경로
file_path = os.path.join(crawled_folder_path, f'naver_blog_{current_datetime}.xlsx')
file_path

## 🔧 블로그 선택
- 참고 링크 : https://section.blog.naver.com/Search/Blog.naver?pageNo=1&orderBy=sim&keyword=아이폰 16
- 참고 링크 : https://section.blog.naver.com/Search/Blog.naver?pageNo=1&orderBy=recentdate&keyword=아이폰 16
- [⭕] orderBy 옵션
    - 관련도순 : &orderBy=sim
    - 최신순 : &orderBy=recentdate

- 수집할 페이지 옵션 추가
- 수집할 페이지가 실제 페이지 수보다 많을 경우 예외 처리

In [40]:
keyword = '아이폰 16' # 검색어
tab_option = '블로그' # 포스트, 블로그 선택
# rangetype = 'PERIOD' # 검색 범위(ALL, WEEK, MONTH, PERIOD)
orderby = 'sim' # 정렬 순서(sim, recentdate)

# # rangetype이 PERIOD인 경우 시작일과 종료일 설정 
# startdate = '2025-01-01' # 형식: YYYY-mm-dd(예. 2025-01-01)
# enddate = '2025-01-10' # 형식: YYYY-mm-dd(예. 2025-01-01)
# current_date = datetime.today().strftime('%Y-%m-%d')
# if startdate > enddate:
#     startdate, enddate = enddate, startdate # 시작일이 종료일보다 크면 종료일로 변경
# if startdate > current_date:
#     startdate, enddate = current_date, current_date # 시작일이 현재 날짜보다 크면 현재 날짜로 변경
# if enddate > current_date:
#     enddate = current_date # 종료일이 현재 날짜보다 크면 현재 날짜로 변경

if tab_option == '글':
    if rangetype == 'PERIOD':
        keyword_url = f'https://section.blog.naver.com/Search/Post.naver?pageNo={page_num}&rangeType={rangetype}&orderBy={orderby}&&startDate={startdate}&endDate={enddate}&keyword={keyword}'
    else:
        keyword_url = f'https://section.blog.naver.com/Search/Post.naver?pageNo={page_num}&rangeType={rangetype}&orderBy={orderby}&keyword={keyword}'
elif tab_option == '블로그':
    keyword_url = f'https://section.blog.naver.com/Search/Blog.naver?pageNo={page_num}&orderBy={orderby}&keyword={keyword}'

In [41]:
keyword_url

'https://section.blog.naver.com/Search/Blog.naver?pageNo=9&orderBy=sim&keyword=아이폰 16'

In [42]:
driver.get(keyword_url)
driver.implicitly_wait(10) # 페이지 로드 될 때까지 기다리지만 로드 되는 순간 종료
# driver.maximize_window() # 브라우져 창 최대화

In [43]:
page = driver.page_source
soup = bs(page, 'html.parser')
print(soup.title.text)

네이버 블로그


In [44]:
area_list_search = soup.select_one('div.area_list_search')
list_search_blog = area_list_search.select('div.list_search_blog')
list_search_blog[0]

<div anchor-text="content" bg-move-scroll-on-pop-state="" class="list_search_blog" ng-repeat="blog in blogSearchCtrl.resultList">
<div class="item">
<a bg-nclick="srb*l.blogger" class="name_blog" href="https://blog.naver.com/gim_melting" ng-href="https://blog.naver.com/gim_melting" target="_blank">
<em class="text_blog" ng-bind-html="blog.blogName">강원도 토박이의 말랑한 국내여행</em>
</a>
<p class="blog_intro" ng-bind-html="blog.blogDesc">강원도 춘천 30년 토박이 - 야놀자 노른자클럽 2기 - <strong class="search_keyword">아이폰 16</strong>pro 사용 중 협업문의gim_melting@naver.com</p>
<div class="writer_info">
<a bg-nclick="srb*l.blogger" class="author" href="https://blog.naver.com/gim_melting" ng-href="https://blog.naver.com/gim_melting" target="_blank">
<div class="thumbnail_author">
<img alt="블로거 썸네일" bg-image="https://blogpfthumb-phinf.pstatic.net/MjAyNDEyMTNfNTEg/MDAxNzM0MDE3Nzg2MDQ2.iluFGxfEcLO7VthkRL3t9fsGVsTL1WbCPiqdoPkNyEsg.FiO1CoYrejZNwN9s_ZYzE0Bm6xpwv4UpGY3l4ZWtngUg.PNG/%25EB%2584%25A4%25EC%259D%25B4%25EB%25B2%2584%25E

In [45]:
# 블로그 제목
post_num = 0
text_blog = list_search_blog[post_num].select_one('em.text_blog').text
text_blog

'강원도 토박이의 말랑한 국내여행'

In [46]:
# 블로그 내용
blog_intro = list_search_blog[post_num].select_one('p.blog_intro').text
blog_intro

'강원도 춘천 30년 토박이 - 야놀자 노른자클럽 2기 - 아이폰 16pro 사용 중 협업문의gim_melting@naver.com'

In [47]:
# 글 작성자
name_author = list_search_blog[post_num].select_one('em.name_author').text
name_author

'김멜팅'

In [48]:
# 블로그 링크
blog_link = list_search_blog[post_num].select_one('a.name_blog')['href']
blog_link

'https://blog.naver.com/gim_melting'

In [49]:
# 블로그 제목
post_num = 1
text_blog = list_search_blog[post_num].select_one('em.text_blog').text
text_blog

'합리적 선택을 도와드위요'

In [50]:
# 블로그 내용
blog_intro = list_search_blog[post_num].select_one('p.blog_intro').text
blog_intro

'✔️아이폰16pro 사용 ✔️25.04 결혼준비 중 ✔️협업 문의: seung2435@naver.com'

In [51]:
list_search_blog[post_num].select_one('p.blog_intro').text == ''

False

In [52]:
# 글 작성자
name_author = list_search_blog[post_num].select_one('em.name_author').text
name_author

'위요'

In [53]:
# 블로그 링크
blog_link = list_search_blog[post_num].select_one('a.name_blog')['href']
blog_link

'https://blog.naver.com/seung2435'

In [ ]:
area_list_search = soup.select_one('div.area_list_search')
list_search_blog = area_list_search.select('div.list_search_blog')

In [54]:
# 정보 수집
text_blog_list = []
blog_intro_list = []
name_author_list = []
blog_link_list = []
for blog in list_search_blog:
    text_blog = blog.select_one('em.text_blog').text
    if blog.select_one('p.blog_intro').text == '':
        blog_intro = np.nan
    else:
        blog_intro = blog.select_one('p.blog_intro').text
    name_author = blog.select_one('em.name_author').text
    blog_link = blog.select_one('a.name_blog')['href']
    # print(text_blog, blog_intro, name_author, blog_link)

    text_blog_list.append(text_blog)
    blog_intro_list.append(blog_intro)
    name_author_list.append(name_author)
    blog_link_list.append(blog_link)

print(len(text_blog_list), len(blog_intro_list), len(name_author_list), len(name_blog_list))

10 10 10 63


In [55]:
blog_dict = {
    'text_blog': text_blog_list,
    'blog_intro': blog_intro_list,
    'name_author': name_author_list,
    'blog_link': blog_link_list
}
blog_df = pd.DataFrame(blog_dict)
blog_df

,text_blog,blog_intro,name_author,blog_link
0,강원도 토박이의 말랑한 국내여행,강원도 춘천 30년 토박이 - 야놀자 노른자클럽 2기 - 아이폰 16pro 사용 중...,김멜팅,https://blog.naver.com/gim_melting
1,합리적 선택을 도와드위요,✔️아이폰16pro 사용 ✔️25.04 결혼준비 중 ✔️협업 문의: seung243...,위요,https://blog.naver.com/seung2435
2,아이엠신뢰예요_슈슈모바일,"가입 -인터넷가입 -알뜰폰가입 -S24,플립6,폴드6 -아이폰16 -S24FE -일...",슈슈,https://blog.naver.com/shushu0091
3,대전 계룡 휴대폰 /한결 /쵸파통신/노랑우체통,"갤럭시s24 s24+ 울트라,폴더폰,대전알뜰폰,휴대폰,대전핸드폰,아이폰16가격,갤럭...",쵸파사장,https://blog.naver.com/dnjswogud11
4,빵트립,함께 가성비 좋은 국내 가볼만한 곳을 소개합니다. ·가끔 자유부인인날 갈만한곳도 올...,빵트립,https://blog.naver.com/dlalgus106
5,대구 휴대폰 하면 당근폰!!,대구/경북 대표맛집 * Z폴드블6 * 아이폰16 시리즈 * S24 시리즈 * 폴더블...,carrot_phone,https://blog.naver.com/carrot_phone
6,미스유 어디가YOU,모습을 전달할게요 인천의 매력과 다채로운 모습을 소개합니다 love4830@nave...,미스유,https://blog.naver.com/love4830
7,엄마는 처음인 혜잠의 애개육아,"24년 8월생 루돌이, 7살 반려견 좌충우돌 육아일기✔️아이폰16PRO ✔협업: e...",혜잠,https://blog.naver.com/hjsdaily
8,KT소닉붐직영점,(5단지 상가 다이소 맞은편 KT/365일 연중무휴) 아이폰16 시리즈 사전예약 이...,카멜,https://blog.naver.com/ktneungdong_store
9,가재부부의 IT이야기,소개하는 가재부부입니다. 카메라 : 소니 a7m4 렌즈 : 탐론 28-75 G2 스...,가재부부,https://blog.naver.com/handrer


## 🔧 종합
- 수집할 페이지 옵션 추가
- 수집할 페이지가 실제 페이지 수보다 많을 경우 예외 처리

In [59]:
keyword = '아이폰 16' # 검색어

In [60]:
rangetype = 'PERIOD' # 검색 범위(ALL, WEEK, MONTH, PERIOD)
orderby = 'sim' # 정렬 순서(sim, recentdate)

# rangetype이 PERIOD인 경우 시작일과 종료일 설정 
startdate = '2025-01-01' # 형식: YYYY-mm-dd(예. 2025-01-01)
enddate = '2025-01-10' # 형식: YYYY-mm-dd(예. 2025-01-01)
current_date = datetime.today().strftime('%Y-%m-%d')
if startdate > enddate:
    startdate, enddate = enddate, startdate # 시작일이 종료일보다 크면 종료일로 변경
if startdate > current_date:
    startdate, enddate = current_date, current_date # 시작일이 현재 날짜보다 크면 현재 날짜로 변경
if enddate > current_date:
    enddate = current_date # 종료일이 현재 날짜보다 크면 현재 날짜로 변경

In [71]:
# 현재 날짜
current_date = datetime.today().strftime('%Y%m%d')
# 현재 경로 확인
code_path = os.getcwd().replace('\\', '/')
# 수집한 파일 저장할 폴더 생성
crawled_folder_path = os.path.join(code_path, 'crawled_data', 'naver_blog', current_date)
os.makedirs(crawled_folder_path, exist_ok=True)
# 저장할 파일 경로
current_datetime = datetime.today().strftime('%Y%m%d_%p_%I%M%S')
file_path = os.path.join(crawled_folder_path, f'naver_blog_{current_datetime}.xlsx')

In [ ]:
# 웹드라이버 실행
driver = webdriver.Chrome(options=options)

sheet_name_list = ['글', '블로그']
with pd.ExcelWriter(file_path, engine='openpyxl') as writer:
    for tab_option in sheet_name_list:
        if tab_option == '글':
            title_list = []
            text_list = []
            name_author_list = []
            name_blog_list = []
            date_list = []
            post_link_list = []
            for page_num in range(1, 10):
                if rangetype == 'PERIOD':
                    keyword_url = f'https://section.blog.naver.com/Search/Post.naver?pageNo={page_num}&rangeType={rangetype}&orderBy={orderby}&&startDate={startdate}&endDate={enddate}&keyword={keyword}'
                else:
                    keyword_url = f'https://section.blog.naver.com/Search/Post.naver?pageNo={page_num}&rangeType={rangetype}&orderBy={orderby}&keyword={keyword}'

                driver.get(keyword_url)
                driver.implicitly_wait(10) # 페이지 로드 될 때까지 기다리지만 로드 되는 순간 종료
                time.sleep(random.uniform(1, 3))

                page = driver.page_source
                soup = bs(page, 'html.parser')
                # print(soup.title.text)
                
                area_list_search = soup.select_one('div.area_list_search')
                list_search_post = area_list_search.select('div.list_search_post')

                for post in list_search_post:
                    title = post.select_one('span.title').text
                    text = post.select_one('a.text').text
                    name_author = post.select_one('em.name_author').text
                    name_blog = post.select_one('span.name_blog').text
                    date = post.select_one('span.date').text
                    post_link = post.select_one('a.desc_inner')['href']

                    title_list.append(title)
                    text_list.append(text)
                    name_author_list.append(name_author)
                    name_blog_list.append(name_blog)
                    date_list.append(date)
                    post_link_list.append(post_link)

            # print(len(title_list), len(text_list), len(name_author_list), len(name_blog_list), len(date_list), len(post_link_list))

            # 데이터 프레임 생성
            post_dict = {
                'title': title_list,
                'text': text_list,
                'name_author': name_author_list,
                'name_blog': name_blog_list,
                'date': date_list,
                'post_link': post_link_list
            }
            post_df = pd.DataFrame(post_dict)
            post_df.to_excel(writer, sheet_name=tab_option, index=False)
            print(post_df.shape)

        elif tab_option == '블로그':
            text_blog_list = []
            blog_intro_list = []
            name_author_list = []
            blog_link_list = []
            for page_num in range(1, 10):
                keyword_url = f'https://section.blog.naver.com/Search/Blog.naver?pageNo={page_num}&orderBy={orderby}&keyword={keyword}'

                driver.get(keyword_url)
                driver.implicitly_wait(10) # 페이지 로드 될 때까지 기다리지만 로드 되는 순간 종료
                time.sleep(random.uniform(1, 3))

                page = driver.page_source
                soup = bs(page, 'html.parser')
                # print(soup.title.text)

                area_list_search = soup.select_one('div.area_list_search')
                list_search_blog = area_list_search.select('div.list_search_blog')

                for blog in list_search_blog:
                    text_blog = blog.select_one('em.text_blog').text
                    if blog.select_one('p.blog_intro').text == '':
                        blog_intro = np.nan
                    else:
                        blog_intro = blog.select_one('p.blog_intro').text
                    name_author = blog.select_one('em.name_author').text
                    blog_link = blog.select_one('a.name_blog')['href']
                    # print(text_blog, blog_intro, name_author, blog_link)

                    text_blog_list.append(text_blog)
                    blog_intro_list.append(blog_intro)
                    name_author_list.append(name_author)
                    blog_link_list.append(blog_link)

                # print(len(text_blog_list), len(blog_intro_list), len(name_author_list), len(name_blog_list))
            blog_dict = {
                'text_blog': text_blog_list,
                'blog_intro': blog_intro_list,
                'name_author': name_author_list,
                'blog_link': blog_link_list
            }
            blog_df = pd.DataFrame(blog_dict)
            blog_df.to_excel(writer, sheet_name=tab_option, index=False)
            print(blog_df.shape)

    print('저장 파일 경로 :', file_path)
    print('저장완료')

63 63 63 63 63 63
10 10 10 63
20 20 20 63
30 30 30 63
40 40 40 63
50 50 50 63
60 60 60 63
70 70 70 63
80 80 80 63
90 90 90 63
저장 파일 경로 : c:/Users/kbjoo/Documents/Google_Drive/GitHub/my_projects/crawling/naver\crawled_data\naver_blog\20250113\naver_blog_20250113_AM_113540.xlsx
저장완료


In [73]:
driver.quit()

# END